In [5]:
import pandas as pd
import numpy as np
import os  
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, classification_report


In [2]:
def filepath(f):
    d = os.path.join(os.path.dirname(os.getcwd()), 'processed_data', f)
    return d

In [3]:
#Import training and test datasets
train_data=pd.read_csv(filepath("final_training_set.csv"))
#test_data=pd.read_csv(filepath("final_test_set.csv"))

In [4]:
train_data["PotentialFraud"].value_counts()

No     345415
Yes    212796
Name: PotentialFraud, dtype: int64

In [55]:
test_data.columns

Index(['PotentialFraud', 'Gender', 'Race', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke',
       'admitDiagInFinalDiagnosis', 'diagnosis_1', 'diagnosis_2',
       'diagnosis_3', 'diagnosis_4', 'diagnosis_5', 'diagnosis_6',
       'diagnosis_7', 'diagnosis_8', 'diagnosis_9', 'diagnosis_10',
       'diagnosis_11', 'diagnosis_12', 'diagnosis_13', 'diagnosis_14',
       'diagnosis_15', 'diagnosis_16', 'diagnosis_17', 'diagnosis_18',
       'diagnosis_19', 'procedure_1', 'procedure_2', 'procedure_3',
       'procedure_4', 'procedure_5', 'procedure_6', 'procedure_7',
       'procedure_8', 'procedure_9', 'procedure_10', 'procedure_11',
       'procedure_12', 'procedure_13', 'procedure_14', 'procedure_15',
      

In [5]:
# List of columns to exclude from the model
#columns_to_exclude = ['Provider', 'BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt','AdmissionDt', 'DischargeDt', 'DOB', 'DOD','diagnosis_descriptor','procedures_descriptor']
columns_to_include = ['PotentialFraud', 'Gender', 'Race', 'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'admitDiagInFinalDiagnosis', 'diagnosis_1',
       'diagnosis_2', 'diagnosis_3', 'diagnosis_4', 'diagnosis_5',
       'diagnosis_6', 'diagnosis_7', 'diagnosis_8', 'diagnosis_9',
       'diagnosis_10', 'diagnosis_11', 'diagnosis_12', 'diagnosis_13',
       'diagnosis_14', 'diagnosis_15', 'diagnosis_16', 'diagnosis_17',
       'diagnosis_18', 'diagnosis_19', 'procedure_1', 'procedure_2',
       'procedure_3', 'procedure_4', 'procedure_5', 'procedure_6',
       'procedure_7', 'procedure_8', 'procedure_9', 'procedure_10',
       'procedure_11', 'procedure_12', 'procedure_13', 'procedure_14',
       'procedure_15', 'procedure_16', 'procedure_17', 'procedure_18']

#train_data = train_data.drop(columns=columns_to_exclude)
#train_data = train_data.drop(columns='RenalDiseaseIndicator')
#train_data = train_data.drop(columns='DeductibleAmtPaid')
#test_data = test_data.drop(columns=columns_to_exclude)

train_data = train_data[columns_to_include]
#test_data = test_data[columns_to_include]

In [6]:
#Define variables needed
target = 'PotentialFraud'
features = [col for col in train_data.columns if col != target]


In [7]:
X_train = train_data[features]
#X_train['DeductibleAmtPaid'].fillna(0, inplace=True)
y_train = train_data[target]

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [8]:
def filepath(f):
    d = os.path.join(os.path.dirname(os.getcwd()), 'processed_data', f)
    return d
#Import training and test datasets
train_data=pd.read_csv(filepath("final_training_set.csv"))
test_data=pd.read_csv(filepath("final_test_set.csv"))
train_data
uid = train_data["ClaimID"]
train_data = train_data.drop(
    ["ClaimID"],
    axis = 1
)
for c in train_data.columns:
    print(c)
def minmax_encode(df, col):
    """
    Return dataset including the minmax encoded column and excluding the original column

    Constraints:
    - col must be a String
    - df must be a Pandas Dataframe
    - df[col] must be numeric
    """

    maxx = df[col].max()
    minx = df[col].min()
    out = list(map(lambda x: (x-minx)/(maxx-minx), df[col]))
    new_colname = col + "_minmax"
    df[new_colname] = out
    return df.drop(
        [col],
        axis = 1
    )
def one_hot_encode(df, col):
    """
    Returns the dataset including the one hot encoded columns and excluding the original column

    Constraints:
    - col must be a String
    - df must be a Pandas Dataframe
    - df[col] must be a Series that represents a categorical variable
    """
    ohe_cols = pd.get_dummies(df[col], prefix = col)
    output = pd.concat(
        [df, ohe_cols],
        axis = 1,
    ).drop(
        [col],
        axis = 1
    )
    return output

def frequency_encode(df, col):
    """
    Returns the dataset including the frequency encoded column and excluding the original column

    Constraints:
    - col must be a String
    - df must be a Pandas Dataframe
    - df[col] must be a Series that represents a categorical variable with high cardinality
    """
    val_counts = df[col].value_counts().to_dict()
    total = len(col)
    out = []
    for x in df[col]:
        out.append(val_counts[x]/total)
    new_colname = col + '_freq'
    df[new_colname] = out
    df.drop(
        [col],
        axis = 1,
        inplace = True
    )

    return df
freq_encoded_cols = []
ohe_cols = []
num_cols = []
unique_threshold = 30

for col in train_data.columns:
    if train_data[col].nunique() == 1:
        print(col,"has been removed as it is constant")
        train_data.drop([col], axis=1, inplace=True)
    elif train_data[col].nunique() == 2: # Binary columns
        continue
    elif train_data[col].dtype in ['int64','float64']:
        train_data[col] = train_data[col].fillna(train_data[col].median())
        num_cols.append(col)
    elif train_data[col].nunique() > unique_threshold:
        freq_encoded_cols.append(col)
    elif 2 < train_data[col].nunique() <= unique_threshold:
        ohe_cols.append(col)
        

for col in train_data.columns:
    if col in num_cols:
        train_data = minmax_encode(train_data, col)
    elif col in ohe_cols:
        train_data = one_hot_encode(train_data, col)
    elif col in freq_encoded_cols:
        try:
            train_data = frequency_encode(train_data, col)
        except:
            print(col)
    elif train_data[col].nunique() == 1:
        train_data.drop(
            [col],
            axis = 1,
            inplace = True
        )
    else:
        train_data[col] = train_data[col].astype('bool')
y = train_data["PotentialFraud"]
train_data.drop(
    ["PotentialFraud"],
    axis = 1,
    inplace = True
)
X_train, X_valid, y_train, y_valid = train_test_split(train_data, y, test_size=0.2, random_state=42)

Provider
PotentialFraud
InscClaimAmtReimbursed
DeductibleAmtPaid
inpatient
Gender
Race
RenalDiseaseIndicator
ChronicCond_Alzheimer
ChronicCond_Heartfailure
ChronicCond_KidneyDisease
ChronicCond_Cancer
ChronicCond_ObstrPulmonary
ChronicCond_Depression
ChronicCond_Diabetes
ChronicCond_IschemicHeart
ChronicCond_Osteoporasis
ChronicCond_rheumatoidarthritis
ChronicCond_stroke
IPAnnualReimbursementAmt
IPAnnualDeductibleAmt
OPAnnualReimbursementAmt
OPAnnualDeductibleAmt
is_alive
age
claim_duration
time_under_care
admitDiagInFinalDiagnosis
diagnosis_1
diagnosis_2
diagnosis_3
diagnosis_4
diagnosis_5
diagnosis_6
diagnosis_7
diagnosis_8
diagnosis_9
diagnosis_10
diagnosis_11
diagnosis_12
diagnosis_13
diagnosis_14
diagnosis_15
diagnosis_16
diagnosis_17
diagnosis_18
diagnosis_19
procedure_1
procedure_2
procedure_3
procedure_4
procedure_5
procedure_6
procedure_7
procedure_8
procedure_9
procedure_10
procedure_11
procedure_12
procedure_13
procedure_14
procedure_15
procedure_16
procedure_17
procedure_18

In [9]:
# Initialize and train the Isolation Forest model
model = IsolationForest(contamination=0.15,random_state=42)

# Define the parameter grid for tuning
param_grid = {
    'n_estimators': [50, 100],  # Reducing the number of trees
    'max_samples': [0.1, 0.5, 1.0],  # Adjust the proportion of samples to use
    'contamination': [0.05, 0.1, 0.2],  # Vary the contamination parameter
}

# Create a grid search cross-validation object
#grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1',n_jobs=-1)

# Fit the grid search to your data
#grid_search.fit(X_train)

# Get the best hyperparameters
#best_params = grid_search.best_params_
#best_model = IsolationForest(random_state=42, n_jobs=-1, **best_params)

#model.fit(X_train)

In [ ]:
#X_test = test_set[features]
#X_test = test_data.drop(columns='DeductibleAmtPaid')
#test_data.loc[:, 'PotentialFraud'] = 0
#y_test = test_set[target]

In [16]:
df = pd.DataFrame(
    {
        "a":['yes','no']
    }
)
df['a'].astype(bool)

0    True
1    True
Name: a, dtype: bool

In [10]:
# Fit the model on the training data
model.fit(X_train)

# Predict on the test set
y_pred_iso = model.predict(X_valid)

# The Isolation Forest model assigns -1 to anomalies (fraud) and 1 to normal data hence we convert -1 to 0
#y_pred_binary = [1 if label == -1 else 0 for label in y_pred]
#y_pred_binary = ["Yes" if label == 1 else "No" for label in y_pred_binary]

y_pred_iso = [1 if label == -1 else 0 for label in y_pred_iso]
y_pred_iso = ["Yes" if label == 1 else "No" for label in y_pred_iso]

In [14]:
print(y_pred_iso)
print(y_valid)

['No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', '

In [12]:
#Evaulating Isolation Forest accuracy
# Evaluate the model on training data
train_accuracy = accuracy_score(y_valid, y_pred_iso)
train_report = classification_report(y_valid, y_pred_iso)

print(f'Training Accuracy: {train_accuracy}')
print(train_report)

c:\Users\SOS05\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:226: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


ValueError: Mix of label input types (string and number)

In [7]:
from lazypredict.Supervised import LazyClassifier 
clf = LazyClassifier(verbose = 0, ignore_warnings=True, custom_metric=None)
lazy_model,lazy_predictions = clf.fit(X_train,X_valid,y_train,y_valid)

 59%|█████▊    | 17/29 [07:18<06:25, 32.10s/it]

In [7]:
from sklearn.linear_model import LogisticRegression

# Create a Logistic Regression model
logistic_model = LogisticRegression(solver='lbfgs', max_iter=1000)

# Train the model on the training data
logistic_model.fit(X_train, y_train)

# Make predictions on the test data
log_y_pred = logistic_model.predict(X_valid)

#log_y_pred = [1 if label == -1 else 0 for label in log_y_pred]
#log_y_pred = ["Yes" if label == 1 else "No" for label in log_y_pred]

ValueError: could not convert string to float: 'PRV52303'

In [19]:
print(log_y_pred)
print(y_valid)

['No' 'No' 'No' ... 'No' 'No' 'Yes']
230744    Yes
27826      No
314625    Yes
140256     No
502955     No
         ... 
512790     No
529291    Yes
215808    Yes
389946     No
293880     No
Name: PotentialFraud, Length: 111643, dtype: object


In [36]:
log_train_accuracy = accuracy_score(y_valid, log_y_pred)
log_train_report = classification_report(y_valid, log_y_pred)

print(f'Training Accuracy: {log_train_accuracy}')
print(log_train_report)

Training Accuracy: 0.627706170561522
              precision    recall  f1-score   support

          No       0.63      0.97      0.76     69258
         Yes       0.59      0.06      0.11     42385

    accuracy                           0.63    111643
   macro avg       0.61      0.52      0.44    111643
weighted avg       0.61      0.63      0.52    111643



In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Create a Random Forest Classifier instance
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=0)

# Fit the model on the training data
rf_classifier.fit(X_train, y_train)

# Predict using the trained model
rf_y_pred = rf_classifier.predict(X_valid)

ValueError: could not convert string to float: 'PRV52303'

In [45]:
rf_test_accuracy = accuracy_score(y_valid, rf_y_pred)
rf_test_report = classification_report(y_valid, rf_y_pred)

print(f'Testing Accuracy: {rf_test_accuracy}')
print(rf_test_report)

Training Accuracy: 0.5905072418333438
              precision    recall  f1-score   support

          No       0.63      0.80      0.71     69258
         Yes       0.43      0.25      0.31     42385

    accuracy                           0.59    111643
   macro avg       0.53      0.52      0.51    111643
weighted avg       0.56      0.59      0.56    111643

